In [1]:
using Pkg
Pkg.activate(".")
using Lux, Random, NNlib, JLD2, DICOM
using Images
using ImageMorphology, Statistics, Dates, imageToolBox
full_data_dir = "/Users/harryxiong24/Code/Lab/Observer/dataset/inter-observer/50_Sohrab_Fati/output"

  Activating project at `~/Code/Lab/Observer/utils`
  Activating new project at `~/Code/Lab/Observer`
  No Changes to `~/Code/Lab/Observer/Project.toml`
  No Changes to `~/Code/Lab/Observer/Manifest.toml`


"/Users/harryxiong24/Code/Lab/Observer/dataset/inter-observer/50_Sohrab_Fati/output"

In [2]:
function keep_largest_component(img; factor=10)
    # resize image
    s = size(img)
    s_ = round.(Int, s ./ factor)
    img_ = round.(imresize(img, s_))

    # Label connected components
    labels = label_components(img_)

    # Count the number of pixels in each component
    component_sizes = [sum(labels .== i) for i in 1:maximum(labels)]

    # Find the label of the largest component
    largest_label = argmax(component_sizes)

    # Return a binary image with only the largest component
    img_ = labels .== largest_label

    # resize image
    img_ = round.(imresize(img_, s))

    return img_
end

keep_largest_component (generic function with 1 method)

In [4]:
threshold = 0.8
SIDs = readdir(full_data_dir)[2:end]

Threads.@threads for i = size(SIDs)[1]:-1:1
    SID = SIDs[i]

    dir1 = joinpath(full_data_dir, SID, "source")
    for f in readdir(dir1)
        f_name, f_ext = splitext(f)
        if f_ext == ".dcm"
            dcm_path = joinpath(dir1, f)
            breast_mask_path = joinpath(dir1, f_name * ".mask.png")

            dcm_data = dcm_parse(dcm_path)
            img = Float32.(dcm_data[(0x7fe0, 0x0010)])
            is_reversed = uppercase(dcm_data[(0x2050, 0x0020)]) == "INVERSE"
            min_, max_ = extrema(img)
            if min_ != 0
                img = (img .- min_) ./ (max_ - min_)
            else
                img = img ./ max_
            end
            if is_reversed
                img = 1.0 .- img
                img = Float32.(img .> threshold)
            else
                img = Float32.(img .> (1 - threshold))
            end
            img = keep_largest_component(img; factor=10)
            # @info breast_mask_path
            save(breast_mask_path, Gray.(img))
            # break
        end
        # break
    end
    GC.gc(true)
    # break
end
GC.gc(true)